## Environments syntax

#### Motivation

- So far we've used pre-defined environments like Frozen Like and Google RecSim.
- To use RL on our own problem, we can't use any of these environments.
- We'll need to define our own environment with Python.

#### Frozen Lake Review

- Recall the Frozen Lake environment, from Module 1:

In [1]:
import gym
env = gym.make("FrozenLake-v1")
env.render()


SFFF
FHFH
FFFH
HFFG


#### Frozen Lake Review

- OpenAI Gym is open source, so we could look at the [Frozen Lake source code](https://github.com/openai/gym/blob/master/gym/envs/toy_text/frozen_lake.py).
- However, it's complicated and contains much more than we need.
- Let's make our own environment called Frozen Pond with the basic components of Frozen Lake.

#### Components of an Env

Conceptual decisions:

- Observation space
- Action space

In Python we will need to implement, at least:

- constructor
- `reset()`
- `step()`

In practice, we may also want other methods, such as `render()`

#### Conceptual decisions

- In this case, since we're mimicking the Frozen Lake, the observation space and action space are already decided.

In [5]:
observation_space = gym.spaces.Discrete(16)
action_space = gym.spaces.Discrete(4)

Later in this course we'll dive deeper into these decisions!

#### Coding it up

In [6]:
import gym

class FrozenPond(gym.Env):
    pass

- Notice that we start by subclassing `gym.Env`.
- Optional: You can read about objects, inheritance, and subclasses.
- Punch line: This is a basic `gym.Env` and we can overwrite features of it.

#### Constructor

- The constructor gets called when we make a new `FrozenPond` object.
- Here is where we define the observation space and action space.

In [7]:
import gym

class FrozenPond(gym.Env):
    def __init__(self):
        self.observation_space = gym.spaces.Discrete(16)
        self.action_space = gym.spaces.Discrete(4)        

#### Reset

- The next method we'll need is reset.


In [ ]:
class FrozenPond(gym.Env):
    def reset(self):
        